In [661]:
import numpy as np

In [662]:
def application_function(f, n_ind, lf, k):
    '''
    ******* Input parameters ********
     # n=[n1,n2,...,nM]  number of BT_blocks at each level, where M is the number of BT levels. 1D np.array with size M, dtype=int
     f                 size of final dense f*f matrix (e.g., f=3 for 3*3 matrix)  scalar, dtype=int
     n_ind             2 x M rectangular matrix indicating the position of each element in the MBT matrix,  2D np.array with size (2,M), dtype=int
     # level             current BT level (level=1,...,M).
     lf                physical length per lattice spacing (length scale factor)
     k                 wavenumber in medium
    *********************************
    ******** Output *****************
    MBT_elem           f x f general matrix of MBT level=M 
    *********************************
    '''
    Xkm= 1.0*(n_ind[0,:]-n_ind[1,:])*lf
    Xkm_abs= np.linalg.norm(Xkm)
    XkmXkm= np.outer(Xkm,Xkm)

    if f == 3 :
        if Xkm_abs == 0 :
            MBT_elem= np.zeros((3,3))
            return MBT_elem
        Gkm= k*k*(np.eye(3)-1j*XkmXkm/Xkm_abs/Xkm_abs)
        Gkm= Gkm-((1.0-1j*k*Xkm_abs)/Xkm_abs/Xkm_abs)*(np.eye(3)-3*XkmXkm/Xkm_abs/Xkm_abs)
        Gkm= Gkm*np.exp(1j*k*Xkm_abs)/Xkm_abs
        MBT_elem= -Gkm
        return MBT_elem
    elif f == 1:
        return 1
    else :
        raise ValueError("f !=3 is unsupported in application function.")


In [663]:
def BT_fft(n,f,n_ind,level,lf,k):
    '''
    ******* Input parameters ********
     n=[n1,n2,...,nM]  number of BT_blocks at each level, where M is the number of BT levels. 1D np.array with size M, dtype=int
     f                 size of final dense f*f matrix (e.g., f=3 for 3*3 matrix) scalar, dtype=int
     n_ind             2 x M rectangular matrix indicating the position of each element in the MBT matrix 2D np.array with size (2,M), dtype=int
     level             current BT level (level=1,...,M).
     lf                physical length per lattice spacing (length scale factor)
     k                 wavenumber in medium
    *********************************
    ******** Output *****************
    MBT_elem           f x f general matrix of MBT level=M in Eigen::MatrixXcd
    *********************************
    '''
    Au= np.zeros(0,dtype=complex)
    if level == len(n)+1 :
        # terminate recursion
        Au.resize(f*f)
        Au= (np.flipud(application_function(f,n_ind,lf,k)).T).flatten()
        return Au
    else :
        Au.resize(f*f*np.prod(2*n[level-1:]-1))
        this_n= n[level-1]
        b_edge= f*f*np.prod(2*n[level:]-1)
        #lower triangular and diagonal blocks
        for i in range(this_n,0,-1):
            n_ind[1,level-1]= 1
            n_ind[0,level-1]= i
            Au[b_edge*(this_n-i):b_edge*(this_n-i+1)]= BT_fft(n,f,n_ind,level+1,lf,k)
        #upper triangular blocks
        for i in range(2,this_n+1,1):
            n_ind[1,level-1]= i
            Au[b_edge*(i+this_n-2):b_edge*(i+this_n-1)]= BT_fft(n,f,n_ind,level+1,lf,k)
        return Au

In [664]:
def BT_pad(n,f,x):
    '''
    Generation of xz by inserting zeros into x
    ****** Input parameters **************
    n=[n1,n2,...,nM]     number of BT_blocks at each level, where M is the number of BT levels. 1D np.array with size M, dtype=int
    f                    size of final dense f*f matrix (e.g., f=3 for 3*3 matrix)
    x                    input vector before padding
    ****** Output ************************
    xz                   padded vector
    **************************************
    '''
    xz= np.zeros(0,dtype=complex)
    lenxz= 0
    if len(n) < 1 :
        lenxz= len(x)+f*f-f
        xz.resize(lenxz)
        xz[0:len(x)]= x
        xz[len(x):lenxz]= np.zeros(lenxz-len(x),dtype=complex)
        return xz
    else :
        b_edge= f*np.prod(n[1:])
        this_n= n[0]

        for i in range(1, this_n+1):
            lenxz_old= len(xz)
            lenblk= len(BT_pad(n[1:],f,x[(i-1)*b_edge:i*b_edge]))
            if len(n) > 1 :
                lenxz= lenxz_old+lenblk+(n[1]-1)*f*f*np.prod(2*n[2:]-1)
                xz= np.pad(xz,(0,lenxz-len(xz)))
                xz[lenxz_old:]= np.zeros(len(xz)-lenxz_old, dtype=complex)
                xz[lenxz_old:lenxz_old+lenblk]= BT_pad(n[1:],f,x[(i-1)*b_edge:i*b_edge])
            else :
                lenxz= lenxz_old+lenblk
                xz= np.pad(xz, (0,lenxz-len(xz)))
                xz[lenxz_old:]= BT_pad(n[1:],f,x[(i-1)*b_edge:i*b_edge])
        return xz

In [665]:
def BT_reconstruct(n,f,bz):
    '''
    Reconstruction of b from bz
    *************************** Input parameters *****************************************
     n=[n1,n2,...,nM]     number of BT_blocks at each level, where M is the number of BT levels. 1D np.array with size M, dtype=int
     f                    size of final dense f*f matrix (e.g., f=3 for 3*3 matrix)
     bz                   input vector
     *************************** Output ***************************************************
     b                    output vector (trimmed)
     ***************************************************************************************/
     '''
    b= np.zeros(0,dtype=complex)
    if len(n) < 1 :
        for i in range(f, f*f+1, f) :
            b= np.pad(b, (0, 1))
            b[-1]= bz[i-1]
        return b
    else :
        b_edge= f*f*np.prod(2*n[1:]-1)
        this_n= n[0]
        for i in range(this_n, 2*this_n) :
            lenb_old= len(b)
            lenblk= len(BT_reconstruct(n[1:],f,bz[(i-1)*b_edge:i*b_edge]))
            lenb= lenb_old+lenblk
            b= np.pad(b,(0,lenb-lenb_old))
            b[lenb_old:lenb]= np.zeros(lenb-lenb_old, dtype=complex)
            b[lenb_old:lenb_old+lenblk]= BT_reconstruct(n[1:],f,bz[(i-1)*b_edge:i*b_edge])
        return b


In [666]:
def MBT_fft_init(n,f,lf,k):
    '''
    Prepare the fourier-transformed MBT projection of MBT matrix
    *************************** Input parameters *****************************************
     n=[n1,n2,...,nM]     number of BT_blocks at each level, where M is the number of BT levels. 1D np.array with size M, dtype=int
     f                    size of final dense f*f matrix (e.g., f=3 for 3*3 matrix)
     lf                   physical length per lattice spacing (length scale factor)
     k                    wavenumber in medium
     *************************** Output ***************************************************
     Au_til                result vector
     ***************************************************************************************/
    '''

    n_ind_init= np.ones((2,len(n)),dtype=int)
    fft_length= f*f*np.prod(2*n-1)

    level_init= 1
    # MBT projection of MBT matrix A
    Au= BT_fft(n,f,n_ind_init,level_init,lf,k)

    if fft_length != len(Au) :
        raise ValueError("Au length wrong!")
    
    Au= Au[::-1] #reverse the order

    # fft using numpy
    Au_til= np.fft.fft(Au)

    return Au_til



In [667]:
n= np.array([2,2,2],dtype=int)
f=3
lf=1
k=1

## MBT projection of the non-diagonal block of DDA matrix
Au_til= MBT_fft_init(n,f,lf,k)


In [668]:
p_hat= np.arange(1, f*np.prod(n)+1, dtype=complex)
p_hat.size

24

In [669]:
p_hatz= BT_pad(n,f,p_hat)
p_hatz.size

162

In [670]:
fft_length= len(Au_til)
p_hatz= np.pad(p_hatz,(0,fft_length-len(p_hatz)))

p_hatz_til= np.fft.fft(p_hatz)
q_hatz_til= Au_til*p_hatz_til
q_hatz= np.fft.ifft(q_hatz_til) 

q_hat= BT_reconstruct(n,f,q_hatz)

q_hat.size

24